# Multi-Agent Collaboration 
---

### What is Multi-Agent Collaboration?
Multi-Agent Collaboration refers to the process where multiple autonomous agents—each capable of independent decision-making—work together to achieve common or complementary objectives. This concept is widely used in fields like artificial intelligence, robotics, distributed computing, and simulation, and it involves several key aspects:

- **Effective Communication and Coordination**:
Agents exchange information and align their actions to collectively achieve a goal, ensuring that tasks are organized and synchronized.

- **Autonomous, Distributed Decision-Making**:
Each agent operates independently, making local decisions while contributing to a broader strategy, which enhances flexibility and fault tolerance.

- **Adaptive Task Specialization**:
Agents focus on specific roles or subtasks based on their capabilities, and they adjust their strategies through iterative feedback, leading to improved overall performance.


### Key Advantages
- **Efficiency Through Task Specialization**:
By assigning specific roles to different agents, the system can handle complex tasks in parallel. This specialization allows each agent to focus on its area of expertise, resulting in faster and more effective problem-solving.

- **Scalability and Flexibility**:
AutoGen's structured communication and dynamic task allocation enable the system to scale easily. It can adapt to varying project complexities by adding or reassigning agents as needed, ensuring that the collaboration remains robust even as demands change.

- **Enhanced Iterative Refinement**:
The framework’s built-in feedback loops and iterative dialogue facilitate continuous improvement. Agents can refine their outputs based on real-time feedback, leading to more accurate and cohesive final results.

**Reference**
- [AutoGen paper: Enabling Next-Gen LLM Applications via Multi-Agent Conversation](https://arxiv.org/abs/2308.08155)
- [Multi-Agent Collabration Concept](https://langchain-ai.github.io/langgraph/concepts/multi_agent/#network) 

In [ ]:
%pip install --upgrade pip -q

In [ ]:
%pip install "autogen-core"==0.4.5 -qU
%pip install "autogen-ext[openai,azure]"==0.4.5 -qU
%pip install "autogen-agentchat"==0.4.5 -qU

In [ ]:
import autogen_agentchat as chat

chat.__version__

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

print(os.getenv('CHAT_MODEL'))

In [ ]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient


project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=os.environ["AIPROJECT_CONNECTION_STRING"]
)

base_url = project_client.inference.get_azure_openai_client(
    api_version="2024-06-01").base_url

api_endpoint = f'https://{base_url.host}/'

api_key = project_client.inference.get_azure_openai_client(
    api_version="2024-06-01").api_key

deployment_name = os.environ["CHAT_MODEL"]

aoai_client = AzureOpenAIChatCompletionClient(
    azure_endpoint=api_endpoint,
    model="gpt-4o-mini",
    azure_deployment=deployment_name,
    api_key=api_key,
    api_version="2024-06-01"
)

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination


# Define a tool
async def get_weather(city: str) -> str:
    return f"The weather in {city} is 73 degrees and Sunny."


async def main() -> None:
    # Define an agent
    weather_agent = AssistantAgent(
        name="weather_agent",
        model_client=aoai_client,
        tools=[get_weather],
    )

    # Define termination condition
    termination = TextMentionTermination("TERMINATE")

    # Define a team
    agent_team = RoundRobinGroupChat(
        [weather_agent], termination_condition=termination)

    # Run the team and stream messages to the console
    stream = agent_team.run_stream(task="What is the weather in New York?")
    await Console(stream)


# NOTE: if running this inside a Python script you'll need to use asyncio.run(main()).
await main()

### Summary
The code snippet above introduces two high level concepts in AgentChat: Agent and Team. An Agent helps us define what actions are taken when a message is received. Specifically, we use the AssistantAgent preset - an agent that can be given access to a model (e.g., LLM) and tools (functions) that it can then use to address tasks. A Team helps us define the rules for how agents interact with each other. In the RoundRobinGroupChat team, agents respond in a sequential round-robin fashion. In this case, we have a single agent, so the same agent is used for each round.